In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np
import os
import datetime
import warnings
warnings.filterwarnings("ignore")

#Plotting
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline


In [2]:
# numerical variables
num_atts = ['bedroomcnt','calculatedbathnbr','age','additional_rooms_count',
           'calculatedfinishedsquarefeet','fullbathcnt','garagecarcnt','garagetotalsqft',
            'latitude','longitude','lotsizesquarefeet', 'roomcnt',
           'numberofstories','structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt','taxamount',
           'N-ValueRatio', 'N-LivingAreaProp', 'N-NonLivingAreaProp','N-ValueProp']

In [3]:
maindir = "/home/anerdi/Desktop/Zillow"

In [4]:
# properties2016 = pd.read_csv(maindir + "/data/properties_2016.csv/properties_2016.csv",usecols =['parcelid',
#                                  'structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt'])
# properties2017 = pd.read_csv(maindir + "/data/properties_2017.csv")

# new_properties = pd.merge(properties2017, 
#         properties2016[['parcelid','structuretaxvaluedollarcnt','taxvaluedollarcnt','landtaxvaluedollarcnt']],
#                          how ='left', on ='parcelid', suffixes=['2017',''])

# assert properties2017.shape[0] == new_properties.shape[0]
# assert (properties2017.parcelid == new_properties.parcelid).all()

# new_properties['structuretaxvaluedollarcnt_diff'] = (-new_properties['structuretaxvaluedollarcnt'] 
#                                                 + new_properties['structuretaxvaluedollarcnt2017'])
# new_properties['taxvaluedollarcnt_diff'] = (-new_properties['taxvaluedollarcnt'] 
#                                                 + new_properties['taxvaluedollarcnt2017'])
# new_properties['landtaxvaluedollarcnt_diff'] = (-new_properties['landtaxvaluedollarcnt'] 
#                                                 + new_properties['landtaxvaluedollarcnt2017'])

# new_properties.to_csv(maindir + "/data/properties_2016_with_2017_tax.csv", index=False)

In [5]:
properties = pd.read_csv(maindir + "/data/properties_2016_with_2017_tax.csv")

#proportion of living area
properties['N-LivingAreaProp'] = properties['calculatedfinishedsquarefeet']/properties['lotsizesquarefeet']

properties['N-NonLivingAreaProp'] = properties['garagetotalsqft']/properties['lotsizesquarefeet']

#Ratio of the built structure value to land area
properties['N-ValueProp'] = properties['structuretaxvaluedollarcnt']/properties['landtaxvaluedollarcnt']

#Ratio of tax of property over parcel
properties['N-ValueRatio'] = properties['taxvaluedollarcnt']/properties['taxamount']

# Pool
properties['poolsizesum'] = properties['poolsizesum'].fillna(0)
# properties['Pool'] = (properties['poolsizesum'] > 0).astype(int)
properties['Pool'] = (properties['pooltypeid2'].fillna(0) + properties['pooltypeid7'].fillna(0)).astype(int)

properties['regionidcounty'] = properties['regionidcounty'].fillna(9999)
properties['regionidcity'] = properties['regionidcity'].fillna(9999)
properties['regionidneighborhood'] = properties['regionidneighborhood'].fillna(9999)
properties['regionidzip'] = properties['regionidzip'].fillna(9999)
properties['typeconstructiontypeid'] = properties['typeconstructiontypeid'].fillna(9999)
properties['airconditioningtypeid'] = properties['airconditioningtypeid'].fillna(9999)
properties['buildingqualitytypeid'] = properties['buildingqualitytypeid'].fillna(9999)
properties['heatingorsystemtypeid'] = properties['heatingorsystemtypeid'].fillna(9999)
properties['propertylandusetypeid'] = properties['propertylandusetypeid'].fillna(9999)
properties['fips'] = properties['fips'].fillna(9999)

# some more feature engineering
properties['age'] = 2017 - properties['yearbuilt']
properties['additional_rooms_count'] = np.maximum((properties['roomcnt'].values 
                                                   - properties['calculatedbathnbr'].values
                                                   - properties['bedroomcnt'].values),0)

# impute missing num_atts per regionid
for countyid in properties.regionidcounty.unique():
    # setup condition
    cond = properties['regionidcounty'] == countyid
    indices = np.where(cond)[0]
    # impute values based on region
    if countyid != 9999:
        properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
                                .fillna(properties.loc[indices,num_atts]
                                .apply(np.mean)))
    else:
        properties.loc[indices,num_atts] = (properties.loc[indices,num_atts]
                                            .fillna(properties[num_atts]
                                            .apply(np.mean)))

### 2016 data

In [6]:
logerror = pd.read_csv(maindir + "/data/train_2016_v2.csv/train_2016_v2.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
logerror['year'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').year)

In [7]:
data2016 = pd.merge(properties,logerror[['parcelid','logerror','month','year','transactiondate']], on='parcelid')
del logerror

In [8]:
data2016.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,N-NonLivingAreaProp,N-ValueProp,N-ValueRatio,Pool,age,additional_rooms_count,logerror,month,year,transactiondate
0,17073783,9999.0,NaN,NaN,2.5,3.0,NaN,9999.0,2.5,NaN,...,0.000000,1.500013,94.148686,0,31.0,0.0,0.0953,1,2016,2016-01-27
1,17088994,9999.0,NaN,NaN,1.0,2.0,NaN,9999.0,1.0,NaN,...,0.056807,1.500042,91.384267,0,27.0,1.0,0.0198,3,2016,2016-03-30
2,17100444,9999.0,NaN,NaN,2.0,3.0,NaN,9999.0,2.0,NaN,...,0.067134,2.361880,77.696055,0,61.0,0.0,0.0060,5,2016,2016-05-27
3,17102429,9999.0,NaN,NaN,1.5,2.0,NaN,9999.0,1.5,NaN,...,0.062162,2.635597,88.095105,0,52.0,1.5,-0.0566,6,2016,2016-06-07
4,17109604,9999.0,NaN,NaN,2.5,4.0,NaN,9999.0,2.5,NaN,...,0.105122,1.000000,90.005784,0,33.0,1.5,0.0573,8,2016,2016-08-08


In [5]:
# data2016.to_csv("/home/anerdi/Desktop/Zillow/data/traindata2016.csv.gz", compression='gzip')

### 2017 data

In [9]:
logerror = pd.read_csv(maindir + "/data/train_2017.csv")
logerror['weeknumber'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').isocalendar()[1])
logerror['month'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').month)
logerror['year'] = logerror['transactiondate'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d').year)

In [10]:
data2017 = pd.merge(properties,logerror[['parcelid','logerror','month','year','transactiondate']], on='parcelid')

In [11]:
del logerror

In [12]:
data2017.head()

,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,N-NonLivingAreaProp,N-ValueProp,N-ValueRatio,Pool,age,additional_rooms_count,logerror,month,year,transactiondate
0,17054981,9999.0,NaN,NaN,5.0,4.0,NaN,9999.0,5.0,NaN,...,0.018366,1.353192,88.876266,1,35.0,2.0,-0.013099,6,2017,2017-06-15
1,17055743,9999.0,NaN,NaN,2.0,3.0,NaN,9999.0,2.0,66.0,...,0.000000,0.428565,77.563371,0,66.0,1.0,0.073985,7,2017,2017-07-26
2,17068109,9999.0,NaN,NaN,1.5,3.0,NaN,9999.0,1.5,66.0,...,0.000000,2.075364,94.499156,0,38.0,1.5,0.071886,7,2017,2017-07-28
3,17073952,9999.0,NaN,NaN,2.0,2.0,NaN,9999.0,2.0,NaN,...,0.056807,0.536145,94.226676,0,28.0,0.0,0.305680,6,2017,2017-06-02
4,17078502,9999.0,NaN,NaN,1.0,2.0,NaN,9999.0,1.0,NaN,...,0.110048,0.334893,93.884129,0,69.0,2.0,-0.073787,7,2017,2017-07-07


In [13]:
data2016.columns == data2017.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True], dtype=bool)

In [14]:
data = data2016.append(data2017)

In [15]:
assert data2016.shape[0] + data2017.shape[0] == data.shape[0]

In [16]:
data.shape

(167888, 75)

In [17]:
data.to_csv("/home/anerdi/Desktop/Zillow/data/traindata20162017_addfeatures.csv.gz", compression='gzip', index=False)